## 기존 Knowledge Base에 Data Source 추가하기

이 노트북에서는 이미 생성된 Amazon Bedrock Knowledge Base에 새로운 Data Source를 추가하는 방법을 시연합니다.

기존 Knowledge Base를 재생성하지 않고 Data Source만 추가하여 새로운 데이터를 색인에 적재할 수 있습니다.

#### 사전 준비 사항:
- 기존에 생성된 Knowledge Base ID
- 데이터가 이미 업로드된 S3 버킷

#### 단계:
1. 기존 Knowledge Base ID 및 S3 버킷 이름 설정
2. Data Source를 Knowledge Base에 추가
3. Ingestion 작업 시작
4. 테스트

## 1. 필요한 라이브러리 불러오기

In [ ]:
%pip install --upgrade pip --quiet
%pip install -r ../requirements.txt --no-deps --quiet
%pip install -r ../requirements.txt --upgrade --quiet

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [ ]:
import time
import boto3
import pprint

pp = pprint.PrettyPrinter(indent=2)

In [ ]:
# Clients
sts_client = boto3.client('sts')
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

print(f"Region: {region}")
print(f"Account ID: {account_id}")

## 2. 기존 Knowledge Base ID 및 S3 버킷 설정

이미 생성된 Knowledge Base의 ID와 데이터가 업로드된 S3 버킷 이름을 입력합니다.

In [ ]:
# 기존 Knowledge Base ID를 입력하세요
existing_kb_id = "YOUR_EXISTING_KB_ID"  # 예: "ABCDEFGHIJ"

# 데이터가 이미 업로드된 S3 버킷 이름을 입력하세요
existing_bucket_name = "YOUR_EXISTING_BUCKET_NAME"  # 예: "my-data-bucket"

print(f"Knowledge Base ID: {existing_kb_id}")
print(f"S3 Bucket: {existing_bucket_name}")

## 3. Knowledge Base 정보 확인

In [ ]:
# Knowledge Base 정보 확인
kb_info = bedrock_agent_client.get_knowledge_base(knowledgeBaseId=existing_kb_id)

print("Knowledge Base 정보:")
print(f"Name: {kb_info['knowledgeBase']['name']}")
print(f"Status: {kb_info['knowledgeBase']['status']}")
print(f"Description: {kb_info['knowledgeBase'].get('description', 'N/A')}")
print(f"Role ARN: {kb_info['knowledgeBase']['roleArn']}")

## 4. Data Source 추가

boto3 클라이언트를 직접 사용하여 새로운 Data Source를 추가합니다.

In [ ]:
# Data Source 이름 생성
data_source_name = f"{existing_kb_id}-additional-s3-{int(time.time())}"

# Data Source 구성
data_source_configuration = {
    "type": "S3",
    "s3Configuration": {
        "bucketArn": f"arn:aws:s3:::{existing_bucket_name}"
    }
}

# 청킹 전략 구성 (기존 KB와 동일하게 설정)
vector_ingestion_configuration = {
    "chunkingConfiguration": {
        "chunkingStrategy": "FIXED_SIZE",
        "fixedSizeChunkingConfiguration": {
            "maxTokens": 300,
            "overlapPercentage": 20
        }
    }
}

print(f"Data Source 이름: {data_source_name}")
print(f"S3 Bucket ARN: arn:aws:s3:::{existing_bucket_name}")

In [ ]:
# Data Source 생성
create_ds_response = bedrock_agent_client.create_data_source(
    name=data_source_name,
    description="Additional data source for existing Knowledge Base",
    knowledgeBaseId=existing_kb_id,
    dataSourceConfiguration=data_source_configuration,
    vectorIngestionConfiguration=vector_ingestion_configuration
)

data_source = create_ds_response["dataSource"]
data_source_id = data_source['dataSourceId']

print("\nData Source 생성 완료:")
print(f"Data Source ID: {data_source_id}")
print(f"Name: {data_source['name']}")
print(f"Status: {data_source['status']}")
pp.pprint(data_source)

## 5. Ingestion 작업 시작

새로 추가된 Data Source에 대해 ingestion 작업을 시작합니다.

In [ ]:
# Ingestion 작업 시작
print(f"Data Source {data_source_id}에 대한 ingestion 작업을 시작합니다...")

start_job_response = bedrock_agent_client.start_ingestion_job(
    knowledgeBaseId=existing_kb_id,
    dataSourceId=data_source_id
)

job = start_job_response["ingestionJob"]
print(f"\nIngestion Job ID: {job['ingestionJobId']}")
print(f"Status: {job['status']}")

In [ ]:
# Ingestion 작업 완료 대기 (선택사항)
print("\nIngestion 작업 상태를 모니터링합니다...")

while True:
    get_job_response = bedrock_agent_client.get_ingestion_job(
        knowledgeBaseId=existing_kb_id,
        dataSourceId=data_source_id,
        ingestionJobId=job['ingestionJobId']
    )
    status = get_job_response["ingestionJob"]["status"]
    print(f"Current status: {status}")
    
    if status in ["COMPLETE", "FAILED"]:
        break
    
    time.sleep(10)

if status == "COMPLETE":
    print("\n✓ Ingestion 작업이 완료되었습니다.")
else:
    print("\n✗ Ingestion 작업이 실패했습니다.")
    print("실패 이유를 확인하려면 AWS 콘솔을 확인하세요.")

## 6. Knowledge Base 테스트

새로운 Data Source가 추가된 Knowledge Base를 테스트합니다.

In [ ]:
query = "AWS의 Security 서비스들에는 어떤 것들이 있습니까?"
foundation_model = "us.amazon.nova-pro-v1:0"

response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': existing_kb_id,
            "modelArn": f"arn:aws:bedrock:{region}:{account_id}:inference-profile/{foundation_model}",
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults": 5
                } 
            }
        }
    }
)

print("응답:")
print(response['output']['text'])

In [ ]:
# 인용 정보 확인
def citations_print(response_ret):
    for num, chunk in enumerate(response_ret, 1):
        print(f'Chunk {num}: ', chunk['content']['text'], end='\n'*2)
        print(f'Chunk {num} Location: ', chunk['location'], end='\n'*2)

citations = response['citations'][0]['retrievedReferences']
print(f"\n사용된 청크 수: {len(citations)}\n")
citations_print(citations)

## 7. 현재 Knowledge Base의 모든 Data Source 확인

In [ ]:
# Knowledge Base의 모든 Data Source 목록 조회
list_ds_response = bedrock_agent_client.list_data_sources(
    knowledgeBaseId=existing_kb_id
)

print(f"Knowledge Base {existing_kb_id}의 Data Source 목록:\n")
for ds_summary in list_ds_response['dataSourceSummaries']:
    print(f"- Data Source ID: {ds_summary['dataSourceId']}")
    print(f"  Name: {ds_summary['name']}")
    print(f"  Status: {ds_summary['status']}")
    print(f"  Updated: {ds_summary.get('updatedAt', 'N/A')}")
    print()

## 정리

이 노트북에서는 기존 Knowledge Base에 새로운 Data Source를 추가하는 간단한 방법을 시연했습니다.

주요 단계:
1. 기존 KB ID와 S3 버킷 이름 설정
2. boto3 클라이언트로 Data Source 직접 생성
3. Ingestion 작업 실행
4. 테스트

이 방법을 사용하면 기존 Knowledge Base를 재생성하지 않고도 새로운 데이터 소스를 추가할 수 있습니다.

**참고:** 기존 KB의 IAM 역할에 새 S3 버킷에 대한 읽기 권한이 이미 있어야 합니다.